In [1]:
from keras.datasets import imdb

# Load the IMDb movie reviews dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data()

# Print the shape of the datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

17464789/17464789 [==============================] - 0s 0us/step
x_train shape: (25000,)
y_train shape: (25000,)
x_test shape: (25000,)
y_test shape: (25000,)


In [2]:
from keras.preprocessing.sequence import pad_sequences

# Set the maximum number of words
max_words = 10000

# Load the IMDb movie reviews dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

# Pad sequences to a maximum length
max_length = 200
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)

# Print the shape of the preprocessed datasets
print("x_train shape after padding:", x_train.shape)
print("x_test shape after padding:", x_test.shape)

x_train shape after padding: (25000, 200)
x_test shape after padding: (25000, 200)


In [3]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define the model
model = Sequential()

# Add an embedding layer
embedding_size = 128
model.add(Embedding(input_dim=max_words, output_dim=embedding_size, input_length=max_length))

# Add an LSTM layer
lstm_units = 64
model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))

# Add a dense layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the RNN model
batch_size = 128
epochs = 5

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
196/196 [==============================] - 168s 838ms/step - loss: 0.4302 - accuracy: 0.7978 - val_loss: 0.3323 - val_accuracy: 0.8622
Epoch 2/5
196/196 [==============================] - 167s 853ms/step - loss: 0.2347 - accuracy: 0.9066 - val_loss: 0.3042 - val_accuracy: 0.8682
Epoch 3/5
196/196 [==============================] - 165s 843ms/step - loss: 0.1706 - accuracy: 0.9374 - val_loss: 0.3381 - val_accuracy: 0.8596
Epoch 4/5
196/196 [==============================] - 157s 799ms/step - loss: 0.1465 - accuracy: 0.9455 - val_loss: 0.3738 - val_accuracy: 0.8642
Epoch 5/5
196/196 [==============================] - 156s 799ms/step - loss: 0.1130 - accuracy: 0.9600 - val_loss: 0.4373 - val_accuracy: 0.8553


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

# Report accuracy
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.optimizers import Adam

# Define the function to create the RNN model
def create_rnn_model(learning_rate=0.001, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=embedding_size, input_length=max_length))
    model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the Keras model with a scikit-learn compatible wrapper
model = KerasClassifier(build_fn=create_rnn_model, verbose=0)

# Define the grid search parameters
param_grid = {
    'learning_rate': [0.01, 0.001],
    'batch_size': [64, 128],
    'optimizer': [Adam]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(x_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

# Define the model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_size, input_length=max_length))
model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model with callbacks
history = model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test), callbacks=[reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Result visualization
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
